# **Notebook: 03 Statistical Analysis**
Contents:
1.

**Imports & settings**

In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import t

PROJECT_ROOT = r'D:/实习/工作/1_遥感检测土壤污染/EAD to GSS'

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)

**Config (years, land-use, paths)**

In [2]:
YEARS = [2020, 2021, 2022, 2023, 2024, 2025]

land_use_types = {
    0: "All",
    7: "Agricultural",
    8: "Residential",
    9: "Industrial",
    11: "Non-developed",
}

QC_DIR = r'Result_hmq/Soil quality data QC/'
OUT_DIR = r'Result_hmq/Statistical Analysis Results/'
QC_DIR = os.path.join(PROJECT_ROOT, QC_DIR)
OUT_DIR = os.path.join(PROJECT_ROOT, OUT_DIR)
os.makedirs(OUT_DIR, exist_ok=True)

# --- Normality & UCL settings ---
ALPHA = 0.05
N_BOOT = 10000
BOOT_RANDOM_STATE = 42

# --- Meta columns (exclude from analytes) ---
META_COLS = [
    "SurveyID", "SiteUID", "SiteObsUID", "PedonUID", "PHUID",
    "PHSampleUID", "PHSubSampleUID", "PHChemicalUID",
    "Longitude", "Latitude", "obsdate", "Location",
]

pd.DataFrame({
    "YEARS": [", ".join(map(str, YEARS))],
    "ALPHA": [ALPHA],
    "N_BOOT": [N_BOOT],
    "QC_DIR": [QC_DIR],
    "OUT_DIR": [OUT_DIR],
})

,YEARS,ALPHA,N_BOOT,QC_DIR,OUT_DIR
0,"2020, 2021, 2022, 2023, 2024, 2025",0.05,10000,D:/实习/工作/1_遥感检测土壤污染/EAD to GSS\Result_hmq/Soil...,D:/实习/工作/1_遥感检测土壤污染/EAD to GSS\Result_hmq/Stat...


**Helper functions (normality + UCL + stats)**

In [3]:
def normality_test_pvalue(x: pd.Series) -> float:
    """
    Shapiro–Wilk normality test p-value (scipy.stats.shapiro)

    Notes:
    - Requires at least 3 observations.
    - Shapiro can be slow/overly sensitive for very large n; we subsample up to 5000.
    """
    x = x.dropna().astype(float)
    n = len(x)
    if n < 3:
        return np.nan

    try:
        from scipy.stats import shapiro
        # safeguard for very large samples
        if n > 5000:
            x = x.sample(n=5000, random_state=42)
        stat, p = shapiro(x.values)
        return float(p)
    except Exception:
        return np.nan


def ucl95_mean_t(x: pd.Series) -> float:
    """One-sided 95% t-based UCL of the mean (normality assumption)."""
    x = x.dropna().astype(float)
    n = len(x)
    if n < 2:
        return np.nan
    mean = x.mean()
    sd = x.std(ddof=1)
    if np.isnan(sd) or sd == 0:
        return float(mean)
    t_crit = t.ppf(0.95, df=n - 1)  # one-sided 95%
    return float(mean + t_crit * sd / np.sqrt(n))


def ucl95_mean_bootstrap(x: pd.Series, n_boot: int = 10000, random_state: int = 42) -> float:
    """
    Nonparametric bootstrap UCL of the mean:
    - resample mean distribution
    - take 95th percentile of bootstrap means (one-sided)
    """
    x = x.dropna().astype(float).values
    n = len(x)
    if n < 2:
        return np.nan

    rng = np.random.default_rng(random_state)
    idx = rng.integers(0, n, size=(n_boot, n))
    boot_means = x[idx].mean(axis=1)
    return float(np.quantile(boot_means, 0.95))


def compute_stats_for_column(x: pd.Series, alpha: float = 0.05, n_boot: int = 10000, random_state: int = 42) -> dict:
    """
    Compute:
    n, mean, median, min, max, sd, p60, p95, UCL95_mean
    Choose UCL method based on normality p-value.
    """
    x_num = x.dropna().astype(float)
    n = len(x_num)

    if n == 0:
        return {
            "n": 0,
            "mean": np.nan, "median": np.nan, "min": np.nan, "max": np.nan,
            "sd": np.nan, "p60": np.nan, "p95": np.nan,
            "normality_p": np.nan, "normality": "NA",
            "UCL95_mean": np.nan, "UCL_method": "NA",
        }

    p_val = normality_test_pvalue(x_num)
    is_normal = (not np.isnan(p_val)) and (p_val > alpha)

    if is_normal:
        ucl = ucl95_mean_t(x_num)
        method = "t-based (normal)"
        normality = "Normal"
    else:
        ucl = ucl95_mean_bootstrap(x_num, n_boot=n_boot, random_state=random_state)
        method = "bootstrap (non-normal)"
        normality = "Non-normal"

    return {
        "n": int(n),
        "mean": float(x_num.mean()),
        "median": float(x_num.median()),
        "min": float(x_num.min()),
        "max": float(x_num.max()),
        "sd": float(x_num.std(ddof=1)) if n >= 2 else np.nan,
        "p60": float(x_num.quantile(0.60)),
        "p95": float(x_num.quantile(0.95)),
        "normality_p": float(p_val) if not np.isnan(p_val) else np.nan,
        "normality": normality,
        "UCL95_mean": float(ucl) if not np.isnan(ucl) else np.nan,
        "UCL_method": method,
    }


def format_value(val):
    """Formatting rule you defined."""
    if pd.isna(val):
        return ""
    if val == 0:
        return "0"
    abs_val = abs(val)
    if abs_val >= 100:
        return f"{val:.0f}"
    elif abs_val >= 10:
        return f"{val:.1f}"
    else:
        return f"{val:.2f}"


**Load one year & preview**

In [4]:
YEAR = 2025
qc_path = os.path.join(QC_DIR, f"Soil quality data QC {YEAR}.xlsx")

processed_data = pd.read_excel(qc_path)
processed_data.head()

,SurveyID,SiteUID,SiteObsUID,PedonUID,PHUID,PHSampleUID,PHSubSampleUID,PHChemicalUID,Longitude,Latitude,obsdate,Location,Aluminium,Antimony,Arsenic,Barium,Beryllium,Boron_aqua,Cadmium,Calcium,Chromium_Total,Cobalt,Copper,Iron_aqua,Lead_aqua,Lithium,Magnesium,Manganese,InorganicMercury,Molybdenum,Nickel_aqua,Phosphorus,Potassium,Selenium_aqua,Silver,Sodium,Strontium,Thallium,Tin,Titanium,Uranium,Vanadium,Zinc,"Nitrogen Kjeldhal, dry mass",satec,satph,TiCl4,TotalOrganicCarbon
0,8,10004161220,3,2,1,1,1,1,54.374728,24.455875,2025-03-03,Abu Dhabi,3773.546,0.193,1.476,28.348,0.1430,14.232,0.111,134070.008,21.776,3.307,7.025,4575.347,3.591,6.108,10655.735,136.361,0.020,0.322,44.780,554.832,1041.435,0.158,5,2811.037,579.463,0.125,0.500,137.116,0.8,9.766,35.144,NaN,27.600,7.8,543.337,1.55
1,8,10004121138,3,2,1,1,1,1,54.363589,24.456906,2025-03-03,Abu Dhabi,3533.754,0.236,1.185,25.252,0.0375,9.633,0.070,94938.068,18.023,2.589,5.987,4067.960,3.535,4.994,7383.653,112.805,0.005,0.275,36.205,689.106,834.345,0.025,5,351.629,336.072,0.125,0.500,97.532,0.8,7.738,80.614,NaN,2.970,8.2,386.484,1.03
2,8,10004222728,3,2,1,1,1,1,54.360639,24.460648,2025-03-03,Abu Dhabi,3809.452,0.275,1.476,30.433,0.1330,9.284,0.311,117007.024,20.796,3.816,7.028,4343.936,15.181,5.450,8886.312,116.821,0.005,0.458,46.199,431.085,829.478,0.061,5,308.117,410.761,0.125,0.500,115.285,0.8,8.383,141.062,NaN,0.984,8.2,456.831,0.83
3,8,10004222572,3,2,1,1,1,1,54.364432,24.461803,2025-03-03,Abu Dhabi,3904.970,NaN,1.418,26.240,0.0375,10.711,0.110,110046.121,21.451,3.432,9.577,4628.338,2.910,5.532,9232.471,144.702,0.005,0.297,50.974,530.253,880.116,0.189,5,385.550,378.814,0.125,1.182,118.783,0.8,8.670,87.112,NaN,1.800,8.2,470.691,1.28
4,8,10004120958,8,2,1,1,1,1,54.358618,24.450317,2025-03-03,Abu Dhabi,3661.223,0.192,1.254,22.432,0.0750,11.098,0.080,87796.806,19.110,3.306,10.042,4198.516,3.051,5.273,7915.330,115.622,0.005,0.297,43.144,502.639,1029.577,0.137,5,2082.510,291.458,0.125,0.500,101.447,0.8,8.020,132.771,NaN,21.700,7.7,401.995,1.21


**Define “run one (year, landuse)” function**

In [5]:
def run_stats_one_year_landuse(
    year: int,
    land_use_type: int,
    qc_dir: str,
    out_dir: str,
    meta_cols_master: list,
    alpha: float = 0.05,
    n_boot: int = 10000,
    random_state: int = 42,
):
    # --- load ---
    qc_path = os.path.join(qc_dir, f"Soil quality data QC {year}.xlsx")
    processed_data = pd.read_excel(qc_path)

    # --- filter ---
    if land_use_type == 0:
        single = processed_data.copy()
    else:
        single = processed_data.loc[processed_data["SurveyID"] == land_use_type].copy()

    # --- meta/analytes ---
    meta_cols = [c for c in meta_cols_master if c in single.columns]
    analyte_cols = [c for c in single.columns if c not in meta_cols]

    # numeric coercion
    for c in analyte_cols:
        single[c] = pd.to_numeric(single[c], errors="coerce")

    df_vals = single.loc[:, analyte_cols].copy()

    # --- compute stats ---
    rows = []
    for col in analyte_cols:
        stats = compute_stats_for_column(
            df_vals[col],
            alpha=alpha,
            n_boot=n_boot,
            random_state=random_state,
        )
        stats["element"] = col
        rows.append(stats)

    summary = pd.DataFrame(rows)[[
        "element", "n",
        "mean", "median", "min", "max", "sd",
        "p60", "p95",
        "UCL95_mean",
        "normality", "normality_p",
        "UCL_method",
    ]]
    summary = summary[summary["n"] > 0].copy()

    # --- normality overview table (audit-friendly) ---
    normality_overview = summary[["element", "n", "normality", "normality_p", "UCL_method"]].copy() \
        .sort_values(["normality", "normality_p"], ascending=[True, True])

    # --- formatted export copy ---
    summary_fmt = summary.copy()
    for c in ["mean", "min", "max", "sd", "median", "p60", "p95", "UCL95_mean"]:
        summary_fmt[c] = summary_fmt[c].apply(format_value)

    # --- export ---
    lu_name = land_use_types.get(land_use_type, "Unknown")
    out_path = os.path.join(out_dir, f"Statistical Analysis Results {year} LU{land_use_type} ({lu_name}).xlsx")

    with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
        summary_fmt.to_excel(writer, sheet_name="Summary_Formatted", index=False)
        summary.to_excel(writer, sheet_name="Summary_Numeric", index=False)
        normality_overview.to_excel(writer, sheet_name="Normality_Audit", index=False)

    # return key artifacts for notebook inspection
    run_info = pd.DataFrame([{
        "year": year,
        "land_use_type": land_use_type,
        "land_use_name": lu_name,
        "n_rows": len(single),
        "n_sites": single["SiteUID"].nunique() if "SiteUID" in single.columns else np.nan,
        "n_analytes": len(analyte_cols),
        "alpha": alpha,
        "n_boot": n_boot,
        "export_path": out_path,
    }])

    return run_info, single, summary, normality_overview


**Run one case & inspect step-by-step**

In [6]:
YEAR = 2025
LandUseType = 0  # 0=All, 7/8/9/11

run_info, Single_use_data, summary_numeric, normality_overview = run_stats_one_year_landuse(
    year=YEAR,
    land_use_type=LandUseType,
    qc_dir=QC_DIR,
    out_dir=OUT_DIR,
    meta_cols_master=META_COLS,
    alpha=ALPHA,
    n_boot=N_BOOT,
    random_state=BOOT_RANDOM_STATE,
)

run_info


C:\Users\blkmo\.conda\envs\myenv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
C:\Users\blkmo\.conda\envs\myenv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


,year,land_use_type,land_use_name,n_rows,n_sites,n_analytes,alpha,n_boot,export_path
0,2025,0,All,709,709,36,0.05,10000,D:/实习/工作/1_遥感检测土壤污染/EAD to GSS\Result_hmq/Stat...


In [7]:
summary_numeric.head(30)

,element,n,mean,median,min,max,sd,p60,p95,UCL95_mean,normality,normality_p,UCL_method
0,Aluminium,706,4301.267041,4061.887075,2562.192607,8203.071129,923.372252,4308.671322,6107.968750,4359.360066,Non-normal,2.951732e-18,bootstrap (non-normal)
1,Antimony,699,0.155388,0.119901,0.042000,0.873000,0.109114,0.136884,0.359500,0.162369,Non-normal,3.501748e-34,bootstrap (non-normal)
2,Arsenic,703,1.757116,1.593415,0.000000,4.113000,0.547574,1.764371,2.798334,1.790912,Non-normal,2.580246e-17,bootstrap (non-normal)
3,Barium,705,27.593246,23.984000,5.017000,125.012000,15.912585,27.195600,58.009443,28.599025,Non-normal,1.022276e-25,bootstrap (non-normal)
4,Beryllium,708,0.108477,0.107776,0.037500,0.273489,0.047381,0.117554,0.186807,0.111417,Non-normal,5.704831e-15,bootstrap (non-normal)
5,Boron_aqua,702,17.397123,13.637000,5.005000,96.417000,12.861917,15.325200,43.840300,18.214729,Non-normal,9.046766e-34,bootstrap (non-normal)
6,Cadmium,703,0.104933,0.099431,0.025000,0.390000,0.050407,0.110384,0.185900,0.108184,Non-normal,4.670478e-17,bootstrap (non-normal)
7,Calcium,707,139720.040468,127210.248373,11646.375460,343567.145069,53763.436865,139975.925760,234236.257800,143107.535395,Non-normal,1.748634e-11,bootstrap (non-normal)
8,Chromium_Total,703,27.795640,22.120377,6.939000,95.764000,16.444037,25.182982,64.226862,28.845615,Non-normal,4.944667e-28,bootstrap (non-normal)
9,Cobalt,700,4.063755,3.004360,0.876000,16.587000,2.817378,3.432400,11.037634,4.244105,Non-normal,1.535137e-32,bootstrap (non-normal)


In [8]:
normality_overview.head(50)


,element,n,normality,normality_p,UCL_method
26,Tin,696,Non-normal,1.313404e-46,bootstrap (non-normal)
16,InorganicMercury,699,Non-normal,3.118449e-42,bootstrap (non-normal)
23,Sodium,696,Non-normal,8.726097e-41,bootstrap (non-normal)
28,Uranium,699,Non-normal,1.447955e-40,bootstrap (non-normal)
12,Lead_aqua,704,Non-normal,4.886157e-40,bootstrap (non-normal)
17,Molybdenum,701,Non-normal,3.178051e-39,bootstrap (non-normal)
32,satec,709,Non-normal,1.587724e-37,bootstrap (non-normal)
30,Zinc,702,Non-normal,4.420603e-36,bootstrap (non-normal)
10,Copper,705,Non-normal,7.254317e-35,bootstrap (non-normal)
1,Antimony,699,Non-normal,3.501748e-34,bootstrap (non-normal)


**Batch run: 2020–2025**

In [9]:
all_runs = []

LANDUSE_RUN_LIST = [0, 7, 8, 9, 11]  # All + four land-use types

for y in YEARS:
    for lu in LANDUSE_RUN_LIST:
        run_info, _, _, _ = run_stats_one_year_landuse(
            year=y,
            land_use_type=lu,
            qc_dir=QC_DIR,
            out_dir=OUT_DIR,
            meta_cols_master=META_COLS,
            alpha=ALPHA,
            n_boot=N_BOOT,
            random_state=BOOT_RANDOM_STATE,
        )
        all_runs.append(run_info)

df_all_runs = pd.concat(all_runs, ignore_index=True)
df_all_runs


C:\Users\blkmo\.conda\envs\myenv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
C:\Users\blkmo\.conda\envs\myenv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
C:\Users\blkmo\.conda\envs\myenv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
C:\Users\blkmo\.conda\envs\myenv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
C:\Users\blkmo\.conda\envs\myenv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:586: UserWarning:

,year,land_use_type,land_use_name,n_rows,n_sites,n_analytes,alpha,n_boot,export_path
0,2020,0,All,295,295,37,0.05,10000,D:/实习/工作/1_遥感检测土壤污染/EAD to GSS\Result_hmq/Stat...
1,2020,7,Agricultural,100,100,37,0.05,10000,D:/实习/工作/1_遥感检测土壤污染/EAD to GSS\Result_hmq/Stat...
2,2020,8,Residential,69,69,37,0.05,10000,D:/实习/工作/1_遥感检测土壤污染/EAD to GSS\Result_hmq/Stat...
3,2020,9,Industrial,116,116,37,0.05,10000,D:/实习/工作/1_遥感检测土壤污染/EAD to GSS\Result_hmq/Stat...
4,2020,11,Non-developed,0,0,37,0.05,10000,D:/实习/工作/1_遥感检测土壤污染/EAD to GSS\Result_hmq/Stat...
5,2021,0,All,365,365,50,0.05,10000,D:/实习/工作/1_遥感检测土壤污染/EAD to GSS\Result_hmq/Stat...
6,2021,7,Agricultural,100,100,50,0.05,10000,D:/实习/工作/1_遥感检测土壤污染/EAD to GSS\Result_hmq/Stat...
7,2021,8,Residential,81,81,50,0.05,10000,D:/实习/工作/1_遥感检测土壤污染/EAD to GSS\Result_hmq/Stat...
8,2021,9,Industrial,169,169,50,0.05,10000,D:/实习/工作/1_遥感检测土壤污染/EAD to GSS\Result_hmq/Stat...
9,2021,11,Non-developed,15,15,50,0.05,10000,D:/实习/工作/1_遥感检测土壤污染/EAD to GSS\Result_hmq/Stat...
